In [0]:
# -------------------------------------------
# Streaming Writer (Delta, availableNow)
# -------------------------------------------
from pyspark.sql.utils import AnalysisException
import traceback

def streaming_write_to_table(df, table_name: str, write_mode: str, job_name: str, source_path: str, query_name: str, logger):
    try:
        if not df.isStreaming:
            logger.warning(f"{job_name} - DataFrame is not streaming, skipping write.")
            return

        checkpoint_path = f"{source_path}/_checkpoints/{job_name}"
        output_path     = f"{source_path}/_bronze_output/{job_name}"

        def log_batch(batch_df, batch_id):
            try:
                cnt = batch_df.count()
                logger.info(f"{job_name} - Batch {batch_id}: {cnt:,} rows")
            except Exception as be:
                logger.warning(f"{job_name} - Batch {batch_id}: log failed: {be}")
                logger.warning(traceback.format_exc())

        q = (df.writeStream
               .format("delta")
               .outputMode(write_mode)
               .option("checkpointLocation", checkpoint_path)
               .queryName(query_name)                 # Enables StreamingQueryListener
               .foreachBatch(log_batch)               # Logs row count
               .trigger(availableNow=True)
               .start(output_path))                   # Writes to path, not table directly
               
        logger.info(f"{job_name} - writeStream started.")
        q.awaitTermination()
        logger.info(f"{job_name} - writeStream finished. Registering table if needed.")

        try:
            spark.table(table_name)  # will succeed if exists
            logger.info(f"{job_name} - Table exists: {table_name}")
        except AnalysisException:
            spark.sql(f"CREATE TABLE IF NOT EXISTS {table_name} USING DELTA LOCATION '{output_path}'")
            logger.info(f"{job_name} - Table registered: {table_name}")
    except Exception as e:
        logger.error(f"{job_name} - Streaming write failed: {e}")
        logger.error(traceback.format_exc())
        raise
